In [2]:
import sys
sys.path.append("../")
from aoc_utils import *

<a id='home'></a>
Each day's work will consist of three tasks:
- **Input**: Parse the day's input file with the function `parse(day, parser, sep)`, which treats the input as a sequence of *entries*, separated by `sep` (default newline); applies `parser` to each entry; and returns the results as a tuple. (Note: `ints` and `atoms` are useful `parser` functions (as are `int` and `str`).)
- **Part 1**: Write code to compute the answer to Part 1, and submit the answer to the AoC site. Use the function `answer` to record the correct answer and serve as a regression test when I re-run the notebook.
- **Part 2**: Repeat coding and `answer` for Part 2.
<br>
1. [Day 1](#day_1)
2. [Day 2](#day_2)
3. [Day 3](#day_3)
4. [Day 4](#day_4)
5. [Day 5](#day_5)
6. [Day 6](#day_6)`
7. [Day 7](#day_7)
8. [Day 8](#day_8)
9. [Day 9](#day_9)
10. [Day 10](#day_10)
11. [Day 11](#day_11)
12. [Day 12](#day_12)
13. [Day 13](#day_13)
14. [Day 14](#day_14)
15. [Day 15](#day_15)

<a id='day_1'></a>
[home](#home)
# [Day 1](https://adventofcode.com/2021/day/1): Sonar Sweep

count the number of times a depth measurement increases from the previous measurement


In [3]:
get_in_file(1,2021)
in1 = parse(1, int)
incr = Counter([1 if b > a else -1 for a, b in pairwise(in1)])[1] 
submit(incr, part="a", day=1, year=2021)

Part a already solved with same answer: 1288


### Part Two

consider sums of a three-measurement sliding window

In [ ]:
sw = [c+b+a for a,b,c in sliding_window(in1, 3)] 
incr = Counter([1 if b > a else -1 for a,b in pairwise(sw)])[1]
submit(incr, part="b", day=1, year=2021)

<a id='day_2'></a>
[home](#home)
# [Day 2](https://adventofcode.com/2021/day/2): Dive!

- forward X increases the horizontal position by X units.
- down X increases the depth by X units.
- up X decreases the depth by X units.


In [ ]:
get_in_file(2,2021)
parse_in = lambda x : { k:int(v) for (k,v) in [str.split(x)] }
in2 = Input(2, parse_in)

In [ ]:
c = Counter()
[c.update(d) for d in in2]; c

In [ ]:
course = c['forward']*(c['down']-c['up'])
submit(course, part="a", day=2, year=2021)

### Part Two

- down X increases your aim by X units.
- up X decreases your aim by X units.
- forward X does two things:
    - It increases your horizontal position by X units.
    - It increases your depth by your aim multiplied by X.

In [ ]:
aim = 0
horiz = 0
depth = 0
for d in in2 :
    for k,v in d.items() :
        if k == 'down':
            aim += v
        if k == 'up':
            aim -= v
        if k == 'forward':
            horiz += v
            depth += aim * v

aim, horiz, depth
submit(horiz*depth, part="b", day=2, year=2021)

<a id='day_3'></a>
[home](#home)
# [Day 3](https://adventofcode.com/2021/day/3): Binary Diagnostic

Each bit in the gamma rate can be determined by finding the most common bit in the corresponding position of all numbers in the diagnostic report

The epsilon rate is calculated in a similar way; rather than use the most common bit, the least common bit from each position is used

The power consumption can then be found by multiplying the gamma rate by the epsilon rate.

In [ ]:
get_in_file(3,2021)
parse_in = lambda x : mapt(int, list(str.strip(x)))
in3 = Input(3, parse_in)

In [ ]:
inT = np.array(in3).T
print(f"input Transposed shape {inT.shape}")
bits_counter = mapt( lambda x : Counter(x.tolist()).most_common(), inT ); bits_counter

In [ ]:
gamma_rate_bits = mapt( lambda x : Counter(x.tolist()).most_common()[0][0], inT )
print (f"gamma_rate_bits {gamma_rate_bits}")
gamma_rate = int("".join(map(str,gamma_rate_bits)), 2); gamma_rate

In [ ]:
epsilon_rate_bits = mapt( lambda x : Counter(x.tolist()).most_common()[-1][0], inT )
print (f"epsilon_rate_bits {epsilon_rate_bits}")
epsilon_rate = int("".join(map(str,epsilon_rate_bits)), 2); epsilon_rate

In [ ]:
submit(gamma_rate*epsilon_rate, part="a", day=3, year=2021)

### Part Two

The bit criteria depends on which type of rating value you want to find:

- To find oxygen generator rating, determine the most common value (0 or 1) in the current bit position, and keep only numbers with that bit in that position. If 0 and 1 are equally common, keep values with a 1 in the position being considered.
- To find CO2 scrubber rating, determine the least common value (0 or 1) in the current bit position, and keep only numbers with that bit in that position. If 0 and 1 are equally common, keep values with a 0 in the position being considered.

In [ ]:

def Oxy_bit_criteria(x) :
    c = Counter(x.tolist())
    #print (c)
    if c[0] == c[1]:
        return 1
    return c.most_common()[0][0]

def CO2_bit_criteria(x) :
    c = Counter(x.tolist())
    #print (c)
    if c[0] == c[1]:
        return 0
    return c.most_common()[-1][0]

def calc_rate(input, bit_fun_criteria) :

    col = 0
    inM = np.array(input)

    while ( inM.shape[0] > 1 ) :
        # find criteria bit in column "col"
        criteria_bit = mapt( bit_fun_criteria, inM.T )[col]
        #print (f"col {col} criteria bit {criteria_bit}")
        # get a boolean vector which select rows that have "most_common_bit" at column "col" 
        row_selector = inM[:,col] == criteria_bit
        # filter all the rows using boolean vector "row_selector" 
        inM = inM[row_selector,:]
        col += 1

    return inM[0]

In [ ]:
oxygen_generator_rating = int("".join(map(str, calc_rate(in3, Oxy_bit_criteria))), 2); oxygen_generator_rating

In [ ]:
co2_scrubber_rating = int("".join(map(str, calc_rate(in3, CO2_bit_criteria))), 2); co2_scrubber_rating

In [ ]:
submit(oxygen_generator_rating*co2_scrubber_rating, part="b", day=3, year=2021)

<a id='day_4'></a>
[home](#home)
# [Day 4](https://adventofcode.com/2021/day/4): Giant Squid

Bingo is played on a set of boards each consisting of a 5x5 grid of numbers. Numbers are chosen at random, and the chosen number is marked on all boards on which it appears. (Numbers may not appear on all boards.) If all numbers in any row or any column of a board are marked, that board wins. (Diagonals don't count.)

In [4]:
get_in_file(4,2021)

in4 = parse(4, ints, sep="\n\n")

def bingo_score(input) :
    order, *boards = input
    boardsM = mapt(lambda b: np.array(b).reshape(5,5), boards)
    boardsK = mapt(lambda b: np.full((5,5), False), boards)
    it_order = iter(order)
    tmpK = list()
    drawn = list()
    bingo = False

    while not bingo :
        draw = next(it_order)
        for bM,bK in zip(boardsM, boardsK) :
            # mask elem is True where "draw" is in bM
            mask = np.isin(bM, draw)
            bK = bK | mask
            # If all numbers in any row or any column of a board are marked
            # that board wins
            if ( np.any(np.all(bK, axis=0)) or 
                 np.any(np.all(bK, axis=1)) ) :
                bingo = True
                break
            
            tmpK.append(bK)
        #
        drawn.append(draw)
        boardsK = tmpK
        tmpK = list()

    return (draw,drawn,bM,bK)

d,D,M,K = bingo_score(in4)
print (M)
#print (D)
#print (K)
#print (np.logical_not(K))
score = np.sum(M[np.logical_not(K)])*d


[[51 18 87 35 55]
 [52 85 79 56 82]
 [83 26 24 29 43]
 [80 76  4 45 13]
 [11 12 99 94 47]]


In [ ]:
submit(score, part="a", day=4, year=2021)

### Part Two

... figure out which board will win last

In [ ]:
def win_boards(input) :
    order, *boards = input
    boardsM = mapt(lambda b: np.array(b).reshape(5,5), boards)
    boardsK = mapt(lambda b: np.full((5,5), False), boards)
    it_order = iter(order)
    tmpK = list()
    tmpM = list()
    drawn = list()
    bingo = False
    bingo_boards = list()

    while True :
        try :
            draw = next(it_order)
            for bM,bK in zip(boardsM, boardsK) :
                # mask elem is True where "draw" is in bM
                mask = np.isin(bM, draw)
                bK = bK | mask
                # If all numbers in any row or any column of a board are marked
                # that board wins
                if ( np.any(np.all(bK, axis=0)) or 
                    np.any(np.all(bK, axis=1)) ) :
                    bingo_boards.append((draw,bM,bK))
                else:
                    tmpK.append(bK)
                    tmpM.append(bM)
            #
            drawn.append(draw)
            boardsK = tmpK
            boardsM = tmpM
            tmpK = list()
            tmpM = list()
        except (StopIteration) :
            break

    return bingo_boards

d,M,K = win_boards(in4)[-1]

score = np.sum(M[np.logical_not(K)])*d

In [ ]:
submit(score, part="b", day=4, year=2021)

<a id='day_5'></a>
[home](#home)
# [Day 5](https://adventofcode.com/2021/day/5): Hydrothermal Venture

In [ ]:
get_in_file(5,2021)

in5 = parse(5, ints)

In [ ]:
class Point(namedtuple('Point', ['x', 'y'])):
    pass

class Segment(namedtuple('Segment', ['p1', 'p2'])):
    def is_V(self):
        return self.p1.x == self.p2.x
    def is_H(self):
        return self.p1.y == self.p2.y
    def is_D(self):
        return (abs(self.p1.x - self.p2.x) == abs(self.p1.y - self.p2.y))
    def min_x(self):
        return min(self.p1.x ,self.p2.x)
    def min_y(self):
        return min(self.p1.y ,self.p2.y)
    def max_x(self):
        return max(self.p1.x ,self.p2.x)
    def max_y(self):
        return max(self.p1.y ,self.p2.y)
    def slope(self):
        return (self.p2.y - self.p1.y)/(self.p2.x - self.p1.x)
    def get_points(self):
        if ( self.is_V() ) :
            return mapt(lambda y: Point(self.p1.x,y), range(self.min_y(),self.max_y()+1))
        if ( self.is_H() ) :
            return mapt(lambda x: Point(x,self.p1.y), range(self.min_x(),self.max_x()+1))
        if ( self.is_D() ) :
            if self.slope() > 0 :
                return mapt(lambda z: Point(z[0],z[1]),
                            zip(range(self.min_x(),self.max_x()+1),
                                range(self.min_y(),self.max_y()+1)))
            else :
                return mapt(lambda z: Point(z[0],z[1]),
                            zip(range(self.max_x(),self.min_x()-1, -1),
                                range(self.min_y(),self.max_y()+1)))


In [ ]:
lines = [Segment(Point(x1,x2),Point(y1,y2)) for (x1,x2,y1,y2) in in5]
HV_lines = [line for line in lines if line.is_H() or line.is_V()]

In [ ]:
maxx = max([seg.max_x() for seg in HV_lines])
maxy = max([seg.max_y() for seg in HV_lines])
maxx,maxy

In [ ]:
pts_cnt = Counter()
mapt(lambda x: pts_cnt.update(Counter(x.get_points())), HV_lines)
# ... number of points where at least two lines overlap ==> >= 2
pts_num = len([v for v in pts_cnt.values() if v > 1])
pts_num = len([v for k,v in pts_cnt.most_common() if v > 1])
pts_num

In [ ]:
submit(pts_num, part="a", day=5, year=2021)

### Part Two

In [ ]:
test1 = [Segment(Point(1,1),Point(3,3)),Segment(Point(4,4),Point(2,2)),
         Segment(Point(0,1),Point(3,4)),Segment(Point(2,3),Point(0,1)),
         Segment(Point(9,7),Point(7,9))]
for t in test1 :
    t.is_D()
    print (t.slope(), t.get_points())


In [ ]:
D_lines = [line for line in lines if line.is_D()]
pts_cnt = Counter()
mapt(lambda x: pts_cnt.update(Counter(x.get_points())), chain(HV_lines,D_lines))
pts_num = len([v for k,v in pts_cnt.most_common() if v > 1])
pts_num

In [ ]:
submit(pts_num, part="b", day=5, year=2021)

<a id='day_6'></a>
[home](#home)
# [Day 6](https://adventofcode.com/2021/day/6): Lanternfish


In [ ]:
get_in_file(6,2021)
in6 = parse(6, ints)

In [ ]:
init_state, = in6
init_test = 3,4,3,1,2

In [ ]:
def population (in_s, days, verbose=False) :
    curr_state = in_s 
    while days > 0 :
        if verbose :
            print (f"...days {days} len(curr_state) = {len(curr_state)}")
        c = Counter(curr_state)
        # Each day, a 0 becomes a 6 and adds a new 8 to the end of the list
        next_state = list(map(lambda x: 6 if x == 0 else x-1 , curr_state))
        # count 0s and append 8s as list
        #next_state += c[0]*[8]
        #next_state.extend(c[0]*[8])
        #curr_state = next_state
        # ------
        curr_state = next_state + c[0]*[8]
        #curr_state = [*next_state, *(c[0]*[8])]
        #curr_state = list(chain(next_state, c[0]*[8]))
        days -= 1
        
    return len(curr_state)

In [ ]:
days = 80
population_80d = population(init_state, days)
print (f"after {days} days there would be {population_80d} lanternfish" )
#days = 256; print (f"after {days} days {population(init_test, days)}" )

In [ ]:
cnt = population(init_state, 80)
submit(cnt, part="a", day=6, year=2021)

### Part Two

How many lanternfish would there be after 256 days?

In [ ]:
def s1(S: list) -> list :
    '''using list comprehention '''
    return [ 6 if x == 0 else x-1 for x in S ]

def s2(S: list) -> list :
    ''' using list generator '''
    return list(( 6 if x == 0 else x-1 for x in S ))

def s3(S: list) -> list :
    ''' using trivial for '''
    res = list()
    for s in S :
        if s == 0 :
            res.append(6)
        else :
            res.append(s-1)
    return res 

def s4(S: list) -> Generator :
    ''' using generator '''
    return ( 6 if x == 0 else x-1 for x in S )

def sc(S: list) -> Iterator :
    ''' '''
    dq = deque()
    #print(f"split {len(S)}")
    for i,c in zip(count_from(),chunks(S,64)):
        #chunk = list(c)
        #print(f"{i}) {chunk} -> {s1(chunk)}")
        dq.append(s4(c))
    return chain.from_iterable(dq)

def population (in_s: list, days: int, S_func: callable =s1, verbose: bool =False) -> int :
    verbose = False
    curr_state = in_s
    print (f"S_func {S_func.__name__} {S_func.__doc__}" )
    while days > 0 :
        if verbose :
            print (f"...days {days} len(curr_state) = {len(curr_state)}")
        c = Counter(curr_state)
        # Each day, a 0 becomes a 6 and adds a new 8 to the end of the list
        #--- OLD ==> too much time for mem allocation/deallocation of huge lists ---#
        next_state = list(S_func(curr_state))
        # count 0s and append 8s list
        curr_state = next_state + c[0]*[8]
        days -= 1
        
    return len(curr_state)

In [ ]:
days = 80
#population2_80d = population(init_test, days, sc)
population2_80d = population(init_state, days, sc)
#assert(population2_80d==population_80d)
print (f"after {days} days there would be {population2_80d} lanternfishes" )

In [ ]:
def better_population (in_s: list, days: int, verbose: bool =False) -> int :

    cS = Counter(in_s)
    days = days
    cnt = 0
    while cnt < days :
        if verbose :
            print (f"day {cnt} {cS.most_common()}")
        # subtract 1 to all Counter keys 
        tmp = Counter()
        for k in cS.keys() :
            if k == 0 :
                tmp[6] += cS[k]
            else :
                tmp[k-1] += cS[k]
        if cS[0] :
            tmp[8] += cS[0]
        cS = tmp        
        cnt += 1

    return cS.total()


In [ ]:
days = 256
better_population_256d = better_population(init_state, days)
print (f"after {days} days there would be {better_population_256d} lanternfishes" )
submit(better_population_256d, part="b", day=6, year=2021)

In [ ]:
%load_ext line_profiler
pname = "lp_population_"
for d in range(140,145,5) :
    %lprun -T lp_file -f s3 -f population population(init_test,d,s4)
    os.rename('lp_file',pname+str(d))
    
%lprun -T lp_file -f better_population better_population(init_state,256)

<a id='day_7'></a>
[home](#home)
# [Day 7](https://adventofcode.com/2021/day/7): The Treachery of Whales

In [ ]:
get_in_file(7,2021)
in7, = parse(7, ints)

In [ ]:
cntr = Counter(in7)
print (f"input size {len(in7)} \
  min,max {min(in7),max(in7)} \
  mean {mean(in7)} \
  median {median(in7)} \
  mode {mode(in7)} \
\n{len(cntr)} {cntr.most_common(10)}")

In [ ]:
for t in (mean(in7),median(in7),mode(in7)):
    print (sum([abs(round(x-t)) for x in in7]))

In [ ]:
fuel4median = sum([abs(round(x-median(in7))) for x in in7])
submit(fuel4median, part="a", day=7, year=2021)

### Part Two

In [ ]:
def best_horiz_pos(pos_list) :
    crab_cntr = Counter(pos_list)
    m,M = min(pos_list),max(pos_list)
    bhp = Counter()
    for target in range(m,M+1) : 
        for crab_pos, crabs in crab_cntr.most_common() :
            bhp[target] += sum(range(abs(target-crab_pos)+1))*crabs

    return bhp


In [ ]:
bhp = best_horiz_pos(in7)
best_crabs_horiz_pos,fuel = bhp.most_common()[-1]
submit(fuel, part="B", day=7, year=2021)

<a id='day_8'></a>
[home](#home)
# [Day 8](https://adventofcode.com/2021/day/8): Seven Segment Search

Each entry consists of ten unique signal patterns, a | delimiter, and finally the four digit output value

In [ ]:
get_in_file(8,2021)
in8 = parse(8,lambda x: x.split("|"))
seg2num = dict(zip([6,2,5,5,4,5,6,3,7,6],list(range(10))))

In [ ]:
out_values = [str.split(y) for x,y in in8 ]

In [ ]:
out_values_2347 = [digit for display_values in out_values for digit in display_values if len(digit) in set({2,4,3,7})]

In [ ]:
submit(len(out_values_2347), part="a", day=8, year=2021)

### Part Two

In [ ]:
def sort_seg(seg :set):
    return  "".join(sorted(seg))

def rule_L6 (L3: set, L4: set, L6: list[set] ) :

    l6 = dict()
    # find 6 in L6 : for each seg_set in L6 if L3 not in seg_Set ==> 6
    for x in L6 :
        if not L3.issubset(x) :
            l6[6] = sort_seg(x)
            #print(x)
            L6.remove(x)
    #print (L6) 
    a,b = L6
    amb = a-b
    if ( amb.issubset(L4) ) :
        l6[9] = sort_seg(a)
        l6[0] = sort_seg(b)
    else :
        l6[0] = sort_seg(a)
        l6[9] = sort_seg(b)

    return l6


def rule_L5 (L2: set, L4: set, L5: list[set] ) :

    l5 = dict()
    # find 3 in L5 : for each seg_set in L5 if L2 in seg_Set ==> 3
    for x in L5 :
        if L2.issubset(x) :
            l5[3] = sort_seg(x)
            #print(x)
            L5.remove(x)
    #print (L5) 
    a,b = L5
    amb = a-b
    if ( amb.issubset(L4) ) :
        l5[5] = sort_seg(a)
        l5[2] = sort_seg(b)
    else :
        l5[2] = sort_seg(a)
        l5[5] = sort_seg(b)
        
    return l5


def display_inference(in_lines):

    pattern_output_lines = [(str.split(x),str.split(y)) for x,y in in_lines ]  
    all_data = list()
    sum_outputs = 0

    for patterns,outputs in pattern_output_lines :
        base = dict()
        other = dict()
        display = dict()
        for pattern in patterns :
            s_pattern = sort_seg(pattern)
            if ( len(s_pattern) in set({2,4,3,7}) ) :
                base[len(s_pattern)] = s_pattern
                display[seg2num[len(s_pattern)]] = s_pattern
            else :
                try : other[len(s_pattern)].append(s_pattern)
                except KeyError : other[len(s_pattern)] = [s_pattern]
        #
        L2 = set(base[2])
        L3 = set(base[3])
        L4 = set(base[4])
        L5 = list(map(set,other[5]))
        L6 = list(map(set,other[6]))
        display.update(rule_L6(L3,L4,L6))
        display.update(rule_L5(L2,L4,L5))
        
        dswap = dict((v,k) for k,v in display.items())
        out_sorted = list(map(sort_seg,outputs))
        sum_outputs = int("".join([str(dswap[osor]) for osor in out_sorted ]))
        
        #all_data.append((base,other,display,dswap,out_sorted))
        all_data.append((display,out_sorted,sum_outputs))
    
    return all_data


In [ ]:

res = 0
for d,ov,s in display_inference(in8) :
    res += s
res    

In [ ]:
submit(res, part="b", day=8, year=2021)

<a id='day_9'></a>
[home](#home)
# [Day 9](https://adventofcode.com/2021/day/9): Smoke Basin
Find the low points - the locations that are lower than any of its adjacent locations.<br>
Most locations have four adjacent locations (up, down, left, and right);<br>
locations on the edge or corner of the map have three or two adjacent locations, respectively.

In [ ]:
get_in_file(9,2021)
p = lambda x: mapt(int,x)
in9 = parse(9,p)

test_in = """\
2199943210
3987894921
9856789892
8767896789
9899965678
"""

in_test = mapt(p,test_in.rstrip().split())


In [ ]:
def get_low_points(inP):
    M = np.array(inP)
    print(M.shape)
    minSet = set()
    for ij in np.ndindex(M.shape):
        adj = get_adj(*ij,M,False)
        #adj = get_adj_(*ij,M)
        if min(adj.values()) > M[ij] :
            # remove any adj(i,j) points from minSet
            minSet.difference_update(adj.keys())
            # add (i,j) point to minSet
            minSet.add(ij)
    low_points = dict()
    for pt in minSet :
        low_points[pt] = M[pt]
    return low_points

In [ ]:
def risk_level(lps: dict) -> int :
    return sum(mapt(lambda x: x+1, lps.values()))

In [ ]:
lp = get_low_points(in9)
print(lp)
risk = risk_level(lp)
risk

In [ ]:
submit(risk, part="a", day=9, year=2021)

### Part Two

A basin is all locations that eventually flow downward to a single low point.<br>
Therefore, every low point has a basin

In [ ]:
def get_basin_points(inP, pt: tuple) -> set:
    M = np.array(inP)
    tmpSet = set()
    tmpSet.add(pt)
    basinSet = set()
    basinSet.add(pt)
    while len(tmpSet) :
        pt = tmpSet.pop()
        adj = get_adj(*pt,M,False)
        for pt in adj :
            if M[pt] < 9 and pt not in basinSet :
                #print(f"add {pt}")
                basinSet.add(pt)
                tmpSet.add(pt)
    return basinSet
        

In [ ]:
get_basin_points(in_test, (0,1))

In [ ]:
for lp in get_low_points(in_test).keys() :
    bp = get_basin_points(in_test, lp)
    print(len(bp),bp)

In [ ]:
bp_len = list()
for lp in get_low_points(in9).keys() :
    bp_len.append(len(get_basin_points(in9, lp)))
bp_len.sort()
bp_len[-3:]


In [ ]:
submit(np.prod(bp_len[-3:]), part="b", day=9, year=2021)

<a id='day_10'></a>
[home](#home)
# [Day 10](https://adventofcode.com/2021/day/10): Syntax Scoring

There are one or more chunks on each line, and chunks contain zero or more other chunks.<br>
Adjacent chunks are not separated by any delimiter;<br>
if one chunk stops, the next chunk (if any) can immediately start.<br>
Every chunk must open and close with one of four legal pairs of matching characters:

- If a chunk opens with (, it must close with ).
- If a chunk opens with [, it must close with ].
- If a chunk opens with {, it must close with }.
- If a chunk opens with <, it must close with >.

A corrupted line is one where a chunk closes with the wrong character - that is, where the characters it opens and closes with do not form one of the four legal pairs listed above.


In [ ]:
get_in_file(10,2021)
in10 = parse(10)

OC = "([{<"
CC = ")]}>"
error_score = {')':3, ']':57, '}':1197, '>':25137 }

test_in = """\
[({(<(())[]>[[{[]{<()<>>
[(()[<>])]({[<{<<[]>>(
{([(<{}[<>[]}>{[]{[(<()>
(((({<>}<{<{<>}{[]{[]{}
[[<[([]))<([[{}[[()]]]
[{[{({}]{}}([{[{{{}}([]
{<[[]]>}<{[{[{[]{()[[[]
[<(<(<(<{}))><([]([]()
<{([([[(<>()){}]>(<<{{
<{([{{}}[<[[[<>{}]]]>[]]
"""

in_test = test_in.rstrip().split()

In [ ]:
def close_chars_idx(line: str) -> list:
    cc_idx = list()
    for cc in CC :
        cc_idx += [i for i, chr in enumerate(line) if chr == cc]
    return sorted(cc_idx)

def open_chars_idx(line: str) -> list:
    oc_idx = list()
    for oc in OC :
        oc_idx += [i for i, chr in enumerate(line) if chr == oc]
    return sorted(oc_idx)

def remove_openclose_chars(idx, line: str) -> str:
    return line[:idx-1]+line[idx+1:]

def open_char(ch):
    return OC[CC.find(ch)]

def close_char(ch):
    return CC[OC.find(ch)]

def corrupted(inP) -> tuple:
    illegal = str()
    expected = str()
    for line in inP :
        # get closing chars indexes
        ccs_idx = close_chars_idx(line)
        while len(ccs_idx) > 0 :
            #print(line)
            #print(ccs_idx)
            # get first closing
            c_idx = ccs_idx.pop(0)
            if line[c_idx-1] != open_char(line[c_idx]) :
                illegal += line[c_idx]
                expected += close_char(line[c_idx-1])
            line = remove_openclose_chars(c_idx,line)
            # adjust closing chars indexes
            ccs_idx = [x-2 for x in ccs_idx]
    return illegal,expected
        

In [ ]:
#ill,expect = corrupted(in_test)
ill,expect = corrupted(in10)
syntax_error_score = sum([error_score[ch]*cnt for (ch,cnt) in Counter(ill).most_common()])

In [ ]:
submit(syntax_error_score, part="a", day=10, year=2021)

### Part Two

Now, discard the corrupted lines. The remaining lines are incomplete.
Incomplete are missing some closing characters at the end of the line.<br>
You just need to figure out the sequence of closing characters that complete all open chunks in the line.<br>

You can only use closing characters (), ], }, or >), and you must add them in the correct order so that only legal pairs are formed and all chunks end up closed.

In [ ]:
def is_line_corrupted(line:str) -> bool:
    ccs_idx = close_chars_idx(line)
    while len(ccs_idx) > 0 :
        #print(line)
        #print(ccs_idx)
        c_idx = ccs_idx.pop(0)
        if line[c_idx-1] != open_char(line[c_idx]) :
            return True
        line = remove_openclose_chars(c_idx,line)
        ccs_idx = [x-2 for x in ccs_idx]
    return False

In [ ]:
#incomplete = list(filter(lambda x: not is_line_corrupted(x), in_test))
incomplete = list(filter(lambda x: not is_line_corrupted(x), in10))

In [ ]:
def complete(inP) -> str:
    missing = list()
    for line in inP :
        ml = str()
        ocs_idx = open_chars_idx(line)
        while len(ocs_idx) > 0 :
            #print(line)
            # get last opening 
            o_idx = ocs_idx.pop()
            # last opening miss its closing
            if line[-1] == line[o_idx] :
                ml += close_char(line[o_idx])
                line = line[:-1]
            else :
                line = remove_openclose_chars(o_idx+1,line)
            #ocs_idx = [x-2 for x in ocs_idx]
        missing.append(ml)
    return missing
    
complete_score = {')':1, ']':2, '}':3, '>':4 }    

In [ ]:
def calc_middle_score(complete_chunks: list) -> int:
    scores = list()
    for cs in complete_chunks :
        ts = 0
        for c in cs :
            ts = ts*5+complete_score[c]
        scores.append(ts)
    scores.sort()
    print(scores)
    return scores[math.floor(len(scores)/2)]


In [ ]:
score = calc_middle_score(complete(incomplete))
score

In [ ]:
submit(score, part="b", day=10, year=2021)

<a id='day_11'></a>
[home](#home)
# [Day 11](https://adventofcode.com/2021/day/11): Dumbo Octopus


You can model the energy levels and flashes of light in steps.<br>
During a single step, the following occurs:
- First, the energy level of each octopus increases by 1.
- Then, any octopus with an energy level greater than 9 flashes.<br>
  This increases the energy level of all adjacent octopuses by 1, including octopuses that are diagonally adjacent.<br>
  If this causes an octopus to have an energy level greater than 9, it also flashes.<br>
  This process continues as long as new octopuses keep having their energy level increased beyond 9. (An octopus can only flash at most once per step.)
- Finally, any octopus that flashed during this step has its energy level set to 0, as it used all of its energy to flash.

In [ ]:
get_in_file(11,2021)
p = lambda x: mapt(int,x)
in11 = parse(11, p)

test_in = """
5483143223
2745854711
5264556173
6141336146
6357385478
4167524645
2176841721
6882881134
4846848554
5283751526
"""
in_test = mapt(p,test_in.rstrip().split())

test_10 = """
0481112976
0031112009
0041112504
0081111406
0099111306
0093511233
0442361130
5532252350
0532250600
0032240000
"""
_10_test = mapt(p,test_10.rstrip().split())

dummy = """
11111
19991
19191
19991
11111
"""
in_dummy = mapt(p,dummy.rstrip().split())


In [ ]:
def get_cells_gt_value(value, inM) :
    # It returns a tuple of arrays one for each dimension.
    # Like in our case it’s a two dimension array, so numpy.where() will returns a tuple of two arrays
    rIdx,cIdx = np.where(inM > value)
    # Length of both the arrays will be same. So to get the list of exact coordinates we can zip these arrays
    return zip(rIdx,cIdx)


In [ ]:
def do_flashing(inP, cnt=1000) :
    M = np.matrix(inP)
    ones = np.ones(M.shape, dtype=int)
    Mn = M
    while cnt > 0 :
        cnt -= 1
        Mn += ones
        flashing = set(get_cells_gt_value(9,Mn))
        flashed = set()
        #print(f"flashing {flashing}")
        while len(flashing) :
            fl = flashing.pop()
            flashed.add(fl)
            adj = get_adj(*fl,Mn)
            #print (f"adj({fl})={adj}")
            for ij in adj.keys() :
                if ij not in flashing and ij not in flashed:
                    Mn[ij] += 1
                    if Mn[ij] > 9 :
                        flashing.add(ij)
        for ij in flashed :
            Mn[ij] = 0
        yield len(flashed)

In [ ]:
g = do_flashing(in11)
#g = do_flashing(in_dummy)

In [ ]:
flashes = 0
for i in range(100) :
    flashes += next(g)
flashes

In [ ]:
submit(flashes, part="a", day=11, year=2021)

### Part Two

What is the first step during which all octopuses flash?

In [ ]:
M = np.matrix(in11)
g = do_flashing(in11)
steps = 1
while next(g) != M.size :
    steps += 1
steps

In [ ]:
submit(steps, part="b", day=11, year=2021)

<a id='day_12'></a>
[home](#home)
# [Day 12](https://adventofcode.com/2021/day/12): Passage Pathing

[Graph_theory](https://en.wikipedia.org/wiki/Graph_theory)
```
    start
    /   \
c--A-----b--d
    \   /
     end

dict({node:set(nodes)})
```
Your goal is to find the number of distinct paths that start at `start`, end at `end`, and don't visit small caves more than once.<br>
So, all paths you find should visit small caves at most once, and can visit big caves any number of times.

In [ ]:
get_in_file(12,2021)
p = lambda x: x.split('-')
in12 = parse(12,p)

t1 = """
start-A
start-b
A-c
A-b
b-d
A-end
b-end
"""
in_t1 = mapt(p,t1.rstrip().split())

t2 = """
dc-end
HN-start
start-kj
dc-start
dc-HN
LN-dc
HN-end
kj-sa
kj-HN
kj-dc
"""
in_t2 = mapt(p,t2.rstrip().split())

def make_graph(inP) :
    _G = defaultdict(set)
    for v1,v2 in inP :
        _G[v1].add(v2)
        _G[v2].add(v1)
    return _G

In [ ]:
def find_all_paths(graph, start, end, path=[]):
    """
    https://www.python.org/doc/essays/graphs/
    add condition str.isupper(node) for visit big caves any number of times
    """
    path = path + [start]
    if start == end:
        return [path]
    if start not in graph:
        return []
    paths = []
    for node in graph[start]:
        if node not in path or str.isupper(node):
            newpaths = find_all_paths(graph, node, end, path)
            for newpath in newpaths:
                paths.append(newpath)
    return paths

In [ ]:
G = make_graph(in12)
paths = find_all_paths(G,"start","end")
len(paths)

In [ ]:
submit(len(paths), part="a", day=12, year=2021)

### Part Two

Big caves can be visited any number of times<br>
A single small cave can be visited at most twice, and the remaining small caves can be visited at most once.<br>
However, the caves named `start` and `end` can only be visited exactly once each:
 - once you leave the `start` cave, you may not return to it
 - once you reach the `end` cave, the path must end immediately
 

In [ ]:
def check_lower(path, node):
    if node in ['start','end'] :
        return False
    low_nodes = Counter([n for n in path if n.islower() and n not in ['start','end']])
    #print(f"node {node} low node {low_nodes}")
    [(k,c)] = low_nodes.most_common(1)
    # return True ==> add the lower node to path
    # if no lower node in path has cnt >= 2 
    return c < 2

def find_all_paths(graph, start, end, path=[]):
    """
    https://www.python.org/doc/essays/graphs/
    
    """
    path = path + [start]
    if start == end:
        return [path]
    if start not in graph:
        return []
    paths = []
    for node in graph[start]:
        if node not in path or str.isupper(node) or check_lower(path,node):
            newpaths = find_all_paths(graph, node, end, path)
            for newpath in newpaths:
                paths.append(newpath)
    return paths

In [ ]:
#G = make_graph(in_t1)
#G = make_graph(in_t2)
G = make_graph(in12)
paths = find_all_paths(G,"start","end")
len(paths) #, paths

In [ ]:
submit(len(paths), part="b", day=12, year=2021)

<a id='day_13'></a>
[home](#home)
# [Day 13](https://adventofcode.com/2021/day/13): Transparent Origami

In [ ]:
get_in_file(13)
dots_str, fold_str = parse(13,sep="\n\n")

dots = mapt(ints, dots_str.rstrip().split("\n"))
fold_instr = [(x[0],int(x[1])) for x in re.findall("(\w)=(\d+)",fold_str)] 

In [ ]:
maxx = max([p[0] for p in dots])
maxy = max([p[1] for p in dots])
dotsM = np.zeros((maxy+1,maxx+1),dtype=np.int8)
for (x,y) in dots :
    dotsM[y][x] = 1
dotsM.shape


In [ ]:
def sym_x(i,j,x):
    return (i,-j+2*x)

def sym_y(i,j,y):
    return (-i+2*y,j)

def fold (M, direction:str, value):
    I,J = M.shape
    if direction == "x":
        rows = I
        cols = value
        sym_fuc = sym_x
    elif direction == "y":
        rows = value
        cols = J
        sym_fuc = sym_y
    else :
        raise RuntimeError 

    newM = np.zeros((rows,cols), dtype=np.int8)
    for i in range(rows) :
        for j in range(cols) :
            try : sym = M[sym_fuc(i,j,value)]
            except IndexError: sym = 0
            newM[i][j] = 1 if M[i][j] + sym > 0 else 0

    return newM 

In [ ]:
folded = fold(dotsM, *fold_instr[0])
(folded>0).sum()

In [ ]:
submit((folded>0).sum(), part="a", day=13, year=2021)

### Part Two

In [ ]:
dq = deque(fold_instr)
to_fold = dotsM
while len(dq) :
    instr = dq.popleft()
    to_fold = fold(to_fold, *instr)
    print(instr)
    print(to_fold.shape)


In [ ]:
to_fold

In [ ]:
np.hsplit(to_fold,8)

In [ ]:
submit("CJCKBRPB", part="b", day=13, year=2021)

<a id='day_14'></a>
[home](#home)
# [Day 14](https://adventofcode.com/2021/day/14): Extended Polymerization

The first line is the polymer template - this is the starting point of the process.
The following section defines the pair insertion rules.</br>
A rule like AB -> C means that when elements A and B are immediately adjacent, element C should be inserted between them.<br/>
These insertions all happen simultaneously.

In [ ]:
get_in_file(14)
polymer_template, rules_str = parse(14,sep="\n\n")

t1 = """\
NNCB

CH -> B
HH -> N
CB -> H
NH -> C
HB -> C
HC -> B
HN -> C
NN -> C
BH -> H
NC -> B
NB -> B
BN -> B
BB -> N
BC -> B
CC -> N
CN -> C    
"""
polymer_template, rules_str = t1.rstrip().split("\n\n")

rules_poly = dict([r.split(" -> ") for r in rules_str.rstrip().split("\n")])

In [ ]:
def polymer_gen(templ_poly: str, rules : dict, step):
    templ = templ_poly
    polymer = ""
    cnt = 0
    while cnt < step :
        cnt += 1
        polymer = templ[0]
        for p1,p2 in pairwise(templ):
            k = "".join((p1,p2))
            polymer += "".join((rules[k],p2))
        print(f"step {cnt} len(polymer) {len(polymer)}")
        yield polymer
        templ = polymer    


In [ ]:
poly_cnt = Counter(list(polymer_gen(polymer_template, rules_poly, 10))[-1])
poly_cnt

In [ ]:
(M,C),(m,c)= poly_cnt.most_common()[0],poly_cnt.most_common()[-1]
C,c

In [ ]:
submit(C-c, part="a", day=14, year=2021)

### Part Two

In [ ]:
def next_polymer(start_poly: str, rules : dict):
    polymer = start_poly[0]
    for p1,p2 in pairwise(start_poly):
        k = "".join((p1,p2))
        polymer += "".join((rules[k],p2))
    print(f"len(polymer) {len(polymer)}")
    return polymer    

In [ ]:
polymer = polymer_template
for i in range(30) :
    polymer = next_polymer(polymer, rules_poly)
    
poly_cnt = Counter(polymer)
poly_cnt

In [ ]:
%load_ext line_profiler
pname = "lp_polimer_"
#for d in range(20,25) :
#    %lprun -T lp_file -f s3 -f population population(init_test,d,s4)
#    os.rename('lp_file',pname+str(d))
    
%lprun -T lp_polimer -f get_polymer get_polymer(polymer_template, rules_poly, 23)

<a id='day_15'></a>
[home](#home)
# [Day 15](https://adventofcode.com/2021/day/15): Chiton

You start in the top left position, your destination is the bottom right position, and you cannot move diagonally.<br>
The number at each position is its risk level;<br>
to determine the total risk of an entire path, add up the risk levels of each position you enter<br>
(that is, don't count the risk level of your starting position unless you enter it; leaving it adds no risk to your total)<br>
Your goal is to find a path with the lowest total risk

In [ ]:
get_in_file(15)
p = lambda x: mapt(int,x)
in15 = parse(15,p)

test_in = """\
1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581
"""
test_in = """\
123
456
789
"""
in_test = mapt(p,test_in.rstrip().split())
testM = np.matrix(in_test)

In [ ]:
from anytree import Node, RenderTree, AsciiStyle, Resolver, ResolverError

def build_tree(start : tuple, M) -> Node:
    auxM = np.zeros(M.shape,dtype=np.int8)
    auxM[start] = M[start]
    dq = deque()
    root = Node(str(start), ij=start, risk=0, sum_risk=0)
    dq.append(root)
    while len(dq) > 0 :
        curr_node = dq.pop()
        #print(f'curr node {curr_node}')
        adj = get_adj(*curr_node.ij,M,False)
        #print(f'adj {adj}')
        m = min(adj.values())
        for ij,risk in adj.items():
            nodes_name = "/".join([""]+[str(node.name) for node in curr_node.path])
            #print(nodes_name)
            if str(ij) not in nodes_name :
                n = Node(str(ij), ij=ij, risk=risk, sum_risk=risk+curr_node.risk, parent=curr_node)
                dq.append(n)
    
    return root
        

In [ ]:
root = build_tree((0,0), testM)
print(RenderTree(root, style=AsciiStyle()))

In [ ]:
test_in = """\
123
456
789
"""

in_test = mapt(p,test_in.rstrip().split())
M = np.array(in_test)
for ij in np.ndindex(M.shape):
    print(f'ij {ij} M[ij]={M[ij]}')
    adj = get_adj(*ij,M,False)
    print(f'adj {adj}')
